# VacationPy
---



In [33]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [34]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,margaret river,-33.9500,115.0667,11.70,88,11,0.95,AU,1718123570
1,1,turinskaya sloboda,57.6232,64.3858,15.85,71,53,1.89,RU,1718123571
2,2,grytviken,-54.2811,-36.5092,-6.70,79,20,5.48,GS,1718123571
3,3,banda aceh,5.5577,95.3222,27.63,82,95,5.53,ID,1718123571
4,4,tinghir,31.5157,-5.5343,28.30,23,44,4.83,MA,1718123571


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [35]:
# Configure the map
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    size = "Humidity",
    tiles = "OSM",
    color = "magenta",
    alpha = 0.5,

    frame_width = 800,
    hover_cols=['City', 'Humidity', 'Max Temp', 'Cloudiness', 'Wind Speed', 'Country'],
    frame_height = 600  
      
)

# Display the map plot
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,City,Max Temp,Cloudiness,Wind Speed,Country)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [36]:
ideal_weather_cities = city_data_df[
    (city_data_df['Max Temp'] >= 18) & (city_data_df['Max Temp'] <= 26) &
    (city_data_df['Humidity'] >= 60) & (city_data_df['Humidity'] <= 90) &
    (city_data_df['Wind Speed'] <= 15)
]

# Drop any rows with null values
ideal_weather_cities = ideal_weather_cities.dropna()

# Display sample data
ideal_weather_cities

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
5,5,adamstown,-25.0660,-130.1015,22.59,79,100,6.51,PN,1718123571
7,7,cidreira,-30.1811,-50.2056,19.02,84,52,4.62,BR,1718123571
8,8,bethel,41.3712,-73.4140,22.19,60,100,2.06,US,1718123540
9,9,larache,35.1932,-6.1557,20.64,67,4,4.13,MA,1718123548
14,14,wailua homesteads,22.0669,-159.3780,24.36,80,20,7.72,US,1718123572
...,...,...,...,...,...,...,...,...,...,...
548,548,souillac,-20.5167,57.5167,24.21,83,75,6.69,MU,1718123633
549,549,isla vista,34.4133,-119.8610,23.52,83,100,0.00,US,1718123633
553,553,ambovombe,-25.1667,46.0833,18.65,79,99,8.28,MG,1718123634
555,555,howard springs,-12.4970,131.0470,19.95,90,44,1.75,AU,1718123634


### Step 3: Create a new DataFrame called `hotel_df`.

In [37]:

hotel_df = ideal_weather_cities.copy()
hotel_df = hotel_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']]

# Add an empty column, "Hotel Name," to the DataFrame
hotel_df['Hotel Name'] = ""
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
5,adamstown,PN,-25.0660,-130.1015,79,
7,cidreira,BR,-30.1811,-50.2056,84,
8,bethel,US,41.3712,-73.4140,60,
9,larache,MA,35.1932,-6.1557,67,
14,wailua homesteads,US,22.0669,-159.3780,80,
...,...,...,...,...,...,...
548,souillac,MU,-20.5167,57.5167,83,
549,isla vista,US,34.4133,-119.8610,83,
553,ambovombe,MG,-25.1667,46.0833,79,
555,howard springs,AU,-12.4970,131.0470,90,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [38]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey":geoapify_key,
    "format":"json",
    "categories":"accommodation.hotel",
    "limit": 1
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame

    latitude = row["Lat"] 
    longitude = row["Lng"]


    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty

    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = response.json()
    
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search


adamstown - nearest hotel: No hotel found
cidreira - nearest hotel: Hotel Castelo
bethel - nearest hotel: Hampton Inn Danbury
larache - nearest hotel: Hotel Tanger
wailua homesteads - nearest hotel: Hilton Garden Inn Kauai Wailua Bay
pacific grove - nearest hotel: Pacific Grove Inn
carnarvon - nearest hotel: No hotel found
antonio enes - nearest hotel: Hotel Quirimbas
albany - nearest hotel: No hotel found
san antonio de pale - nearest hotel: No hotel found
hasaki - nearest hotel: Choshi Plaza Hotel
macas - nearest hotel: Nivel 5
port mathurin - nearest hotel: Escale Vacances
la passe - nearest hotel: L'Estuaire
xai-xai - nearest hotel: No hotel found
sorong - nearest hotel: Marina Mamberamo
avarua - nearest hotel: Paradise Inn
nova vicosa - nearest hotel: Pousada Pontal da Barra
lao cai - nearest hotel: Nhà Nghỉ Hoàng Hôn
conception bay south - nearest hotel: No hotel found
ribeira grande - nearest hotel: Hospedaria JSF
imbituba - nearest hotel: Silvestre Praia Hotel
ribeira brava - n

,City,Country,Lat,Lng,Humidity,Hotel Name
5,adamstown,PN,-25.0660,-130.1015,79,No hotel found
7,cidreira,BR,-30.1811,-50.2056,84,Hotel Castelo
8,bethel,US,41.3712,-73.4140,60,Hampton Inn Danbury
9,larache,MA,35.1932,-6.1557,67,Hotel Tanger
14,wailua homesteads,US,22.0669,-159.3780,80,Hilton Garden Inn Kauai Wailua Bay
...,...,...,...,...,...,...
548,souillac,MU,-20.5167,57.5167,83,Hotel SkylineInn
549,isla vista,US,34.4133,-119.8610,83,The Club & Guest House
553,ambovombe,MG,-25.1667,46.0833,79,La source
555,howard springs,AU,-12.4970,131.0470,90,Rydges Palmerston


In [39]:
# Configure the map
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    
    tiles = "OSM",
    color = "red",
    alpha = 0.5,

    frame_width = 800,
    hover_cols=['City', 'Country', 'Humidity', 'Cloudiness', 'Hotel Name'],
    frame_height = 600  
      
)

# Display the map plot
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Country,Humidity,Hotel Name)